## Montage du drive


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)
root = "/content/gdrive/MyDrive/Stage_Bilbao_Neuroblastoma"
!unzip {root}/DataBase/250x250_split_BA.zip &> /dev/null

## Hyper-paramètres

In [ ]:
backbone = "ResNet"
version  = 152

lr = 1e-4
batch_size = 16
optimizer_name = "SGD"

# Do epochs iterations or reach acc_goal on training dataset
# To only do epochs iterations, set train_acc_goal > 1
epochs = 25
train_acc_goal = 1.1

# experiments saved in {backbone}{version}_{optimizer}_{tag}
TAG = "UFN10"

NUM_SESSION = 10
BEST_LOSS = 0.278
BEST_ACCU = 0.883

## Imports python

In [ ]:
!pip install torchmetrics &> /dev/null

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torchmetrics.classification as classification
import copy
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import glob

from torch import nn
from tqdm import tqdm
from torchvision import models
from google.colab import runtime

from torchvision.models.vgg import vgg11, VGG11_Weights, vgg13, VGG13_Weights, vgg16, VGG16_Weights, vgg19, VGG19_Weights
from torchvision.models import inception_v3, Inception_V3_Weights

## Chargement des datasets et preprocessing

In [ ]:
root_dir = "."
train_dir = "train/"
valid_dir = "valid/"
test_dir  = "test/"

if backbone == "Inception":
    input_size = (299, 299)
else:
    input_size = (224, 224)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

train_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Resize(input_size, antialias=True),
    transforms.RandomCrop(input_size),
    transforms.Normalize((0.66747984797634830, 0.5799524696639141, 0.78054363559995920), (0.23162625605944703, 0.2340601507820534, 0.14160506754101998))
    ]
)
eval_transform = transforms. Compose([
    transforms.ToTensor(),
    # transforms.Resize(input_size, antialias=True),
    transforms.RandomCrop(input_size),
    transforms.Normalize((0.66747984797634830, 0.5799524696639141, 0.78054363559995920), (0.23162625605944703, 0.2340601507820534, 0.14160506754101998))
    ]
)

# Chargement des données
train_dataset = datasets.ImageFolder(f"{root_dir}/{train_dir}", train_transform)
valid_dataset = datasets.ImageFolder(f"{root_dir}/{valid_dir}", eval_transform)

print(len(train_dataset), len(valid_dataset))

## Modèle

In [ ]:
def get_classifier(kind: str, in_features: int):
    layers = []
    if kind and kind.startswith("CNL"):
        nb_ReLU = int(kind[-1])
        for i in range(nb_ReLU):
            layers += [
                nn.Linear(in_features >> i, in_features >> (i + 1), bias=True),
                nn.LeakyReLU(0.2, inplace=False),
                nn.Dropout(0.5, inplace=False)
            ]
        in_features = in_features >> (i+1)

    layers += [
        nn.Linear(in_features, 1, bias=True),
        nn.Sigmoid()
    ]
    classifier = nn.Sequential(*layers)

    return classifier

def print_trainable(model):
    total_params, gradient_params = 0, 0
    for param in model.parameters():
        total_params += param.numel()
        if param.requires_grad:
            gradient_params += param.numel()
    print(f"{gradient_params}/{total_params} parameters will be train.")


class ImprovedInception(nn.Module):
    def __init__(self, version=3):
        super(ImprovedInception, self).__init__()
        if version == 3:
            self.inception = inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
        else:
            raise ValueError(f"Inception version {version} not available.")

        self.inception.fc = get_classifier(TAG, in_features=2048)

    def fine_tune(self, unfrozen):
        for param in self.parameters():
            param.requires_grad = False

        layers = list(self.parameters())
        num_layer = len(layers) - 1

        while unfrozen and num_layer:
            layer = layers[num_layer]
            ndim  = len(layer.shape)
            if ndim > 1:
                unfrozen -= 1
            layer.requires_grad = True
            num_layer -= 1

    def forward(self, x):
        return self.inception(x)

    def get_trainable_parameters(self):
        return filter(lambda p: p.requires_grad, self.parameters())


class ImprovedVGG(nn.Module):
    def __init__(self, version=16):
        super(ImprovedVGG, self).__init__()

        if version == 11:
            self.vgg = vgg11(weights=VGG11_Weights.IMAGENET1K_V1)
        elif version == 13:
            self.vgg = vgg13(weights=VGG13_Weights.IMAGENET1K_V1)
        elif version == 16:
            self.vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        elif version == 19:
            self.vgg = vgg19(weights=VGG19_Weights.IMAGENET1K_V1)
        else:
            raise ValueError(f"VGG version {version} does not exist.")

        self.vgg.classifier[6] = get_classifier(TAG, in_features=4096)

    def fine_tune(self, unfrozen):
        for param in self.parameters():
            param.requires_grad = False

        layers = list(self.parameters())
        num_layer = len(layers) - 1

        while unfrozen and num_layer:
            layer = layers[num_layer]
            ndim  = len(layer.shape)
            if ndim > 1:
                unfrozen -= 1
            layer.requires_grad = True
            num_layer -= 1

    def forward(self, x):
        return self.vgg(x)

    def get_trainable_parameters(self):
        return filter(lambda p: p.requires_grad, self.parameters())


class ImprovedResNet(nn.Module):
    def __init__(self, resnet_version=18):
        super(ImprovedResNet, self).__init__()

        if resnet_version == 18:
            self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        elif resnet_version == 34:
            self.resnet = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
        elif resnet_version == 50:
            self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        elif resnet_version == 101:
            self.resnet = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V2)
        elif resnet_version == 152:
            self.resnet = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)
        else:
            raise ValueError(f"ResNet version {resnet_version} does not exist.")

        in_features = 512 if resnet_version <= 34 else 2048
        self.resnet.fc = get_classifier(TAG, in_features)

    def fine_tune(self, unfrozen):
        for param in self.parameters():
            param.requires_grad = False

        layers = list(self.parameters())
        num_layer = len(layers) - 1

        while unfrozen and num_layer:
            layer = layers[num_layer]
            ndim  = len(layer.shape)
            if ndim > 1:
                unfrozen -= 1
            layer.requires_grad = True
            num_layer -= 1

    def forward(self, x):
        return self.resnet(x)

    def get_trainable_parameters(self):
        return filter(lambda p: p.requires_grad, self.parameters())

## Entraînement

In [ ]:
from IPython.utils.path import target_outdated
def train_model(model, loader, criterion, accuracy):
    loss, accu = np.zeros((len(loader))), np.zeros((len(loader)))
    accuracy.reset()
    model.train()
    for b, (inputs, labels) in enumerate(loader):
        (inputs, labels) = (inputs.to(device), labels.to(device))
        labels  = labels.unsqueeze(1).to(torch.float32)
        outputs = model(inputs)
        # outputs, _ = model(inputs)
        cr_loss = criterion(outputs, labels)

        optimizer.zero_grad()
        cr_loss.backward()
        optimizer.step()

        loss[b] = cr_loss.item()
        accu[b] = accuracy(outputs, labels).item()

    mean_loss = np.mean(loss)
    mean_accu = np.mean(accu)
    return mean_loss, mean_accu

def eval_model(model, loader, criterion, accuracy):
    loss, accu = np.zeros((len(loader))), np.zeros((len(loader)))
    accuracy.reset()
    model.eval()
    with torch.no_grad():
        for b, (inputs, labels) in enumerate(loader):
            (inputs, labels) = (inputs.to(device), labels.to(device))
            labels  = labels.unsqueeze(1).to(torch.float32)
            outputs = model(inputs)

            cr_loss = criterion(outputs, labels)
            loss[b] = cr_loss.item()
            accu[b] = accuracy(outputs, labels).item()

    mean_loss = np.mean(loss)
    mean_accu = np.mean(accu)
    return mean_loss, mean_accu

def delete_file(path):
    with open(path, "w") as fd:
        fd.write("")
    os.remove(path)

def find_backup_path(tab_name: str):
    os.makedirs(tab_name, exist_ok=True)
    backup_path = f"{tab_name}/{lr}_{batch_size}"
    os.makedirs(backup_path, exist_ok=True)
    backup_path += "/"
    i = 1
    while os.path.exists(backup_path + str(i)): i += 1
    backup_path += str(i)
    return backup_path

def init_exp(backbone, version):
    def is_tmp_folder(tmp_folder: str):
        try:
            os.makedirs(tmp_folder)
            return False
        except FileExistsError:
            return True

    best_metrics = {"loss": float("inf"), "accu": 0}
    if is_tmp_folder(TMP_PATH):
        H = {
            "train": {"loss": np.load(f"{TMP_PATH}/train_loss.npy"),
                      "accu": np.load(f"{TMP_PATH}/train_accu.npy")},
            "valid": {"loss": np.load(f"{TMP_PATH}/valid_loss.npy"),
                      "accu": np.load(f"{TMP_PATH}/valid_accu.npy")}
            }
        best_metrics["loss"] = np.min(H["valid"]["loss"][np.nonzero(H["valid"]["loss"])])
        best_metrics["accu"] = np.max(H["valid"]["accu"])
        weight_path = glob.glob(os.path.join(TMP_PATH, "weight_*.pth"))[0]
        e = int(weight_path.split("_")[-1].split(".")[0])
        if backbone == "ResNet":
            model = ImprovedResNet(resnet_version=version)
        elif backbone == "VGG":
            model = ImprovedVGG(version)
        elif backbone == "Inception":
            model = ImprovedInception(version)
        else:
            model = None
        model.load_state_dict(torch.load(weight_path))
        # remove residual .pth files
        weight_files = glob.glob(f"{TMP_PATH}/weight_*.pth")
        weight_files.remove(weight_path)
        for w_file in weight_files:
            delete_file(w_file)
    else:
        H = {
            "train": {"loss": np.zeros((epochs+1)),
                      "accu": np.zeros((epochs+1))},
            "valid": {"loss": np.zeros((epochs+1)),
                      "accu": np.zeros((epochs+1))}
            }
        e = 0
        if backbone == "ResNet":
            model = ImprovedResNet(resnet_version=version)
        elif backbone == "VGG":
            model = ImprovedVGG(version)
        elif backbone == "Inception":
            model = ImprovedInception(version)
        else:
            model = None
    model.to(device)
    if TAG and TAG.startswith("UFN"):
        unfrozen = int(TAG[-1]) + 1
    else:
        unfrozen = 1
    model.fine_tune(unfrozen)
    return H, best_metrics, e, model

def save_tmp_exp(H, e, model):
    os.makedirs(TMP_PATH, exist_ok=True)
    np.save(f"{TMP_PATH}/train_loss.npy", H["train"]["loss"])
    np.save(f"{TMP_PATH}/train_accu.npy", H["train"]["accu"])
    np.save(f"{TMP_PATH}/valid_loss.npy", H["valid"]["loss"])
    np.save(f"{TMP_PATH}/valid_accu.npy", H["valid"]["accu"])
    torch.save(model.state_dict(), f"{TMP_PATH}/weight_{e}.pth")
    if e > 1:
      delete_file(f"{TMP_PATH}/weight_{e-1}.pth")

def create_details_dic(train_loader, valid_loader, optimizer_name, lr, batch_size, H):
    details = {}
    details["loader"]   = {"train": len(train_loader), "valid": len(valid_loader)}
    details["learning"] = {"optimizer": optimizer_name, "lr": lr, "batch_size": batch_size, "epoch": len(H["valid"]["loss"])-1}
    details["best"] = {"epoch_loss": int(np.argmin(H["valid"]["loss"])),
                       "epoch_accu": int(np.argmax(H["valid"]["accu"])),
                       "loss": float(np.min(H["valid"]["loss"])),
                       "accu": float(np.max(H["valid"]["accu"]))}
    details["train_loss"] = H["train"]["loss"].tolist()
    details["train_accu"] = H["train"]["accu"].tolist()
    details["valid_loss"] = H["valid"]["loss"].tolist()
    details["valid_accu"] = H["valid"]["accu"].tolist()
    return details

def make_backup_from_tmp(H, details):
    backup_path = find_backup_path(TAB_NAME)
    os.rename(TMP_PATH, backup_path)
    with open(f"{backup_path}/details.json", "w") as fd:
        json.dump(details, fd)
    e = details["learning"]["epoch"]
    plot_metric(H, max_epoch=e, metric="accu")
    plt.savefig(f"{backup_path}/accu.pdf", bbox_inches="tight")
    plot_metric(H, max_epoch=e, metric="loss")
    plt.savefig(f"{backup_path}/loss.pdf", bbox_inches="tight")
    # remove useless files
    for npy_file in ["train_loss", "train_accu", "valid_loss", "valid_accu"]:
        file_ = f"{backup_path}/{npy_file}.npy"
        delete_file(file_)
    weight_files = glob.glob(f"{backup_path}/weight_*.pth")
    for w_file in weight_files:
        delete_file(w_file)

def plot_metric(H, max_epoch:int, metric:str="accu"):
    fig = plt.figure(figsize=(6, 3))
    cmap = plt.get_cmap("tab10")
    colors = [cmap(1), cmap(0)]
    xline = np.arange(max_epoch+1)
    handles = []
    labels = ["valid", "train"]
    for c, dataset in zip(colors, labels):
        mean = H[dataset][f"{metric}"][:max_epoch+1]
        line, = plt.plot(xline, mean, color=c)
        handles += [line]
    plt.xlabel("epoch")
    if metric == "accu":
        plt.ylim([min(min(H["train"]["accu"]), min(H["valid"]["accu"])), 1])
    elif metric == "loss":
        plt.ylim([0, max(max(H["train"][f"loss"]), max(H["valid"][f"loss"]))])
    plt.legend(handles[::-1], labels[::-1])
    plt.grid()

In [ ]:
TAB_NAME = f"{root}/G_Collab/backup/{backbone}{version}_{optimizer_name}"
if TAG: TAB_NAME += f"_{TAG}"
os.makedirs(TAB_NAME, exist_ok=True)
os.makedirs(f"{TAB_NAME}/__Summary__", exist_ok=True)
os.makedirs(f"{TAB_NAME}/{lr}_{batch_size}", exist_ok=True)
TMP_PATH = f"{TAB_NAME}/{lr}_{batch_size}/tmp"

remain_session = NUM_SESSION - len(os.listdir(f"{TAB_NAME}/{lr}_{batch_size}")) + os.path.exists(TMP_PATH)
if TAG:
    print(f"Running {remain_session} sessions for {backbone}{version} (#{TAG})")
else:
    print(f"Running {remain_session} sessions for {backbone}{version}")
print(f"optimized by {optimizer_name} with {lr=} and {batch_size=}.")

for _ in range(remain_session):
    H, best_metrics, e0, model = init_exp(backbone, version)
    best_valid_loss = best_metrics["loss"]
    best_valid_accu = best_metrics["accu"]
    best_epoch_loss, best_epoch_accu = -1, -1

    if optimizer_name == "SGD":
        optimizer = optim.SGD(model.get_trainable_parameters(), lr, momentum=0.9)
    else:
        optimizer = optim.Adam(model.get_trainable_parameters(), lr, betas=(0.9, 0.999))

    criterion = nn.BCELoss()
    accuracy  = classification.BinaryAccuracy().to(device)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size, shuffle=False)

    # Epoch 0
    if e0 == 0:
        (H["train"]["loss"][0], H["train"]["accu"][0]) = eval_model(model, train_loader, criterion, accuracy)
        (H["valid"]["loss"][0], H["valid"]["accu"][0]) = eval_model(model, valid_loader, criterion, accuracy)

    for e in tqdm(range(e0+1, epochs+1)):
        (H["train"]["loss"][e], H["train"]["accu"][e]) = train_model(model, train_loader, criterion, accuracy)
        (H["valid"]["loss"][e], H["valid"]["accu"][e]) = eval_model(model, valid_loader, criterion, accuracy)

        if best_valid_loss > H["valid"]["loss"][e]:
            best_valid_loss = H["valid"]["loss"][e]
            best_epoch_loss = e
            if BEST_LOSS > best_valid_loss:
                torch.save(model.state_dict(), f"{TMP_PATH}/model_loss.pth")
                BEST_LOSS = best_valid_loss
        if best_valid_accu < H["valid"]["accu"][e]:
            best_valid_accu = H["valid"]["accu"][e]
            best_epoch_accu = e
            if BEST_ACCU < best_valid_accu:
                torch.save(model.state_dict(), f"{TMP_PATH}/model_accu.pth")
                BEST_ACCU = best_valid_accu
        save_tmp_exp(H, e, model)
        if H["train"]["accu"][e] > train_acc_goal:
            H["train"]["loss"][e+1:] = None
            H["train"]["accu"][e+1:] = None
            H["valid"]["loss"][e+1:] = None
            H["valid"]["accu"][e+1:] = None
            break

    details = create_details_dic(train_loader, valid_loader, optimizer_name, lr, batch_size, H)
    make_backup_from_tmp(H, details)
runtime.unassign()